<a href="https://colab.research.google.com/github/muhiqbalalamin/DeepLearning/blob/main/Regression_Infrared_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow

In [ ]:
!pip install torch torchvision

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 30.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 23.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 66.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# Load dataset
data_path = '/content/drive/MyDrive/Folder DL/Infrared.csv'
df = pd.read_csv(data_path)

# Handle missing values
df = df.dropna(subset=['Distance'])

# One-hot encoding for categorical features
df = pd.get_dummies(df, columns=['Gender', 'Age', 'Ethnicity'], drop_first=True)

# Split features and target
X = df.drop(columns=['aveOralM'])
y = df['aveOralM'].values.reshape(-1, 1)

# Standardize numerical features
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)

# Normalize target variable
y_scaler = StandardScaler()
y_scaled = y_scaler.fit_transform(y)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y_scaled, test_size=0.2, random_state=42)

# Convert to numpy arrays
X_train, X_test = np.array(X_train), np.array(X_test)
y_train, y_test = np.array(y_train), np.array(y_test)

# PyTorch Model
class DeepModelTorch(nn.Module):
    def __init__(self, input_dim):
        super(DeepModelTorch, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_dim, 64),   # W*x + b
            nn.ReLU(),                  # max(0, x)
            nn.BatchNorm1d(64),         # (x - batch_mean) / (batch_std + epsilon)
            nn.Dropout(0.2),            # Each neuron has 0.2 probability of being zeroed
            nn.Linear(64, 32),          # W*x + b
            nn.ReLU(),                  # max(0, x)
            nn.Linear(32, 1)            # Final output layer
        )

    def forward(self, x):
        return self.model(x)

# Initialize PyTorch model
input_dim = X_train.shape[1]
model_torch = DeepModelTorch(input_dim)
criterion = nn.MSELoss()  # Implements MSE equation
optimizer = optim.Adam(model_torch.parameters(), lr=0.0001)

# Convert data to PyTorch tensors
X_train_torch = torch.tensor(X_train, dtype=torch.float32)
y_train_torch = torch.tensor(y_train, dtype=torch.float32)
X_test_torch = torch.tensor(X_test, dtype=torch.float32)
y_test_torch = torch.tensor(y_test, dtype=torch.float32)

# Training history tracking
torch_losses = []

# Train PyTorch model
for epoch in range(100):
    model_torch.train()  # Set model to training mode
    optimizer.zero_grad()  # Reset gradients

    # Forward pass
    outputs = model_torch(X_train_torch)

    # Calculate loss
    loss = criterion(outputs, y_train_torch)

    # Backward pass (computes ∇θJ(θ))
    loss.backward()

    # Update weights (θ = θ - α * ∇θJ(θ))
    optimizer.step()

    torch_losses.append(loss.item())
    if epoch % 10 == 0:
        print(f'Epoch {epoch}, Loss: {loss.item()}')

print("PyTorch model training complete!")

# Predict with PyTorch model
model_torch.eval()  # Set model to evaluation mode
with torch.no_grad():
    y_pred_torch = model_torch(X_test_torch).numpy()

    # Inverse transform to get original scale
    y_test_original = y_scaler.inverse_transform(y_test)
    y_pred_torch_original = y_scaler.inverse_transform(y_pred_torch)

# TensorFlow Model
model_tf = keras.Sequential([
    layers.Dense(64, activation='relu', input_shape=(X_train.shape[1],)),
    layers.BatchNormalization(),
    layers.Dropout(0.2),
    layers.Dense(32, activation='relu'),
    layers.Dense(1)
])

# Compile TensorFlow model
model_tf.compile(optimizer=keras.optimizers.Adam(learning_rate=0.0001), loss='mse', metrics=['mse'])

# Train TensorFlow model
history = model_tf.fit(
    X_train, y_train,
    epochs=100,
    verbose=1,
    batch_size=32,
    validation_split=0.1
)

# Predict with TensorFlow model
y_pred_tf = model_tf.predict(X_test)
y_pred_tf_original = y_scaler.inverse_transform(y_pred_tf)

def evaluate_model(y_true, y_pred, model_name):
    y_true = y_true.flatten()
    y_pred = y_pred.flatten()

    # Mean Squared Error
    mse = mean_squared_error(y_true, y_pred)

    # Root Mean Squared Error
    rmse = np.sqrt(mse)

    # R-squared (coefficient of determination)
    r2 = r2_score(y_true, y_pred)

    print(f"{model_name} Evaluation:")
    print(f"MSE: {mse:.4f}")
    print(f"RMSE: {rmse:.4f}")
    print(f"R-Squared: {r2:.4f}\n")

    return mse, rmse, r2

# Evaluate both models
torch_metrics = evaluate_model(y_test_original, y_pred_torch_original, "PyTorch Model")
tf_metrics = evaluate_model(y_test_original, y_pred_tf_original, "TensorFlow Model")


Epoch 0, Loss: 1.2308884859085083
Epoch 10, Loss: 1.1031980514526367
Epoch 20, Loss: 0.9842941164970398
Epoch 30, Loss: 0.885841965675354
Epoch 40, Loss: 0.7898789644241333
Epoch 50, Loss: 0.6847495436668396
Epoch 60, Loss: 0.6239565014839172
Epoch 70, Loss: 0.5339627861976624
Epoch 80, Loss: 0.5031497478485107
Epoch 90, Loss: 0.46035173535346985
PyTorch model training complete!


/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 3s 23ms/step - loss: 2.1855 - mse: 2.1855 - val_loss: 1.0993 - val_mse: 1.0993
Epoch 2/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 1.8382 - mse: 1.8382 - val_loss: 0.9059 - val_mse: 0.9059
Epoch 3/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.5370 - mse: 1.5370 - val_loss: 0.7870 - val_mse: 0.7870
Epoch 4/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 1.4590 - mse: 1.4590 - val_loss: 0.7012 - val_mse: 0.7012
Epoch 5/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 1.2181 - mse: 1.2181 - val_loss: 0.6489 - val_mse: 0.6489
Epoch 6/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - loss: 1.0788 - mse: 1.0788 - val_loss: 0.6086 - val_mse: 0.6086
Epoch 7/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.9363 - mse: 0.9363 - val_loss: 0.5874 - val_mse: 0.5874
Epoch 8/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - loss: 0.9195 - mse: 0.9195 - val_loss: 0.5559 - val_mse: 0.5559
Epoch 9/100
23/23 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.9

# Penjelasan metrik evaluasi:
## EQUATION 1: Standardization (Z-score normalization)
For each feature x:

    z = (x - μ) / σ
where:

    z = standardized value
    x = original value
    μ = mean of the feature
    σ = standard deviation of the feature

This transforms features to have mean=0 and standard deviation=1

## EQUATION 2: Neural Network Forward Pass
For each layer l:

    z^(l) = W^(l) * a^(l-1) + b^(l)
    a^(l) = activation(z^(l))
where:

    z^(l) = weighted input to layer l
    W^(l) = weight matrix for layer l
    a^(l-1) = activations from previous layer
    b^(l) = bias vector for layer l
    activation() = activation function (ReLU for hidden layers)


## EQUATION 3: Mean Squared Error Loss

    MSE = (1/n) * Σ(y_i - ŷ_i)²
where:

    n = number of samples
    y_i = true value
    ŷ_i = predicted value


## EQUATION 4: Backpropagation and Gradient Descent

    θ = θ - α * ∇θJ(θ)
where:

    θ = model parameters (weights and biases)
    α = learning rate (0.0001 in our case)
    ∇θJ(θ) = gradient of the loss function with respect to parameters


## EQUATION 5: Batch Normalization

    BN(x) = γ * ((x - μ_B) / sqrt(σ²_B + ε)) + β
where:

    x = input values
    μ_B = mini-batch mean
    σ²_B = mini-batch variance
    γ, β = learnable parameters
    ε = small constant for numerical stability


## EQUATION 6: Evaluation Metrics

MSE (Mean Squared Error):

    MSE = (1/n) * Σ(y_i - ŷ_i)²

RMSE (Root Mean Squared Error):

    RMSE = sqrt(MSE) = sqrt((1/n) * Σ(y_i - ŷ_i)²)

R² (Coefficient of Determination):

    R² = 1 - (SS_res / SS_tot)
    where:
        SS_res = Σ(y_i - ŷ_i)² (Sum of squares of residuals)
        SS_tot = Σ(y_i - y_mean)² (Total sum of squares)